In [10]:
import os
import sys
from importnb import Notebook
import re
import pandas as pd
import numpy as np
import zipfile
from enum import Enum
import zlib

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)
    
with Notebook():
    from src.parserbase import SegBase

In [44]:
class Sav:
    
    def __init__(self, filename:str):
        self.filename = os.path.join(PKG_ROOT, 'saves', 'original', filename)
        self.zipfile = zipfile.ZipFile(self.filename)
        self.filelist = self.zipfile.namelist()
        self.files = [self.zipfile.read(_) for _ in self.filelist]
        self.gam_str = self.files[1].decode('latin')
        self.gam_version = re.findall(r'GAME\s*V\d+\.\d+', self.gam_str)[0]  #always 'GAMEV2.0'
        self.cre_version = re.findall(r'CRE\s*V\d+\.\d+', self.gam_str)[0]  #always 'CRE V1.0'
        
        

In [45]:
a = Sav('pstsave.pstsave')

In [343]:
s = '''<div id="mlevel1">
      SPWI101 Chromatic Orb<br />
      SPWI102 Armor<br />
      SPWI103 Minor Embalming<br />
      SPWI104 Fist of Iron<br />
      SPWI105 Identify<br />
      SPWI107 Magic Missile<br />
      SPWI108 Pacify<br />
      SPWI111 Shield<br />
      SPWI113 Missile of Patience<br />
      SPWI114 Friends<br />
      SPWI115 Blindness<br />
      SPWI116 Tongues of Flame<br />
      SPWI117 Seeking Flames<br />
      SPWI118 Scripture of Steel<br />
      SPWI119 Submerge the Will<br />
    </div>
    <div id="mlevel2">
      SPWI120 Viquar's Eye<br />
      SPWI121 Reign of Anger<br />
      SPWI201 Adder's Kiss<br />
      SPWI202 Black-Barbed Curse<br />
      SPWI203 Black-Barbed Shield<br />
      SPWI204 Blood Bridge<br />
      SPWI205 Greater Embalming<br />
      SPWI207 Ice Knife<br />
      SPWI209 Pain Mirror<br />
      SPWI211 Strength<br />
      SPWI212 Swarm Curse<br />
      SPWI213 Horror<br />
      SPWI214 Luck<br />
      SPWI215 Knock<br />
      SPWI216 Blur<br />
      SPWI217 Ignus' Terror<br />
      SPWI218 Infernal Orb<br />
      SPWI219 Power of One<br />
    </div>
    <div id="mlevel3">
      SPWI301 Ball Lightning<br />
      SPWI304 Cloak of Warding<br />
      SPWI305 Elysium's Tears<br />
      SPWI307 Hold Undead<br />
      SPWI308 Tasha's Unbearable Derisive Laughter<br />
      SPWI309 Vampiric Touch<br />
      SPWI310 Ax of Torment<br />
      SPWI311 Zerthimon's Focus<br />
      SPWI312 Infernal Shield<br />
      SPWI313 Fiery Rain<br />
      SPWI314 Balance in all Things<br />
    </div>
    <div id="mlevel4">
      SPWI401 Blacksphere<br />
      SPWI402 Confusion<br />
      SPWI405 Force Missiles<br />
      SPWI406 Improved Strength<br />
      SPWI408 Shroud of Shadows<br />
      SPWI409 Remove Curse<br />
      SPWI410 Elemental Strike<br />
    </div>
    <div id="mlevel5">
      SPWI501 Cloudkill<br />
      SPWI502 Cone of Cold<br />
      SPWI504 Enoll Eva's Duplication<br />
      SPWI505 Desert Hell<br />
      SPWI506 Fire and Ice<br />
    </div>
    <div id="mlevel6">
      SPWI601 Antimagic Shell<br />
      SPWI602 Globe of Invulnerability<br />
      SPWI603 Howl of Pandemonium<br />
      SPWI604 Chain Lightning Storm<br />
    </div>
    <div id="mlevel7">
      SPWI701 Acid Storm<br />
      SPWI702 Bladestorm<br />
      SPWI704 Guardian Mantle<br />
      SPWI705 Stygian Ice Storm<br />
    </div>
    <div id="mlevel8">
      SPWI802 Meteor Storm Bombardment<br />
      SPWI803 Deathbolt<br />
      SPWI804 Ignus' Fury<br />
      SPWI805 Power Word: Blind<br />
      SPWI807 Mechanus' Cannon<br />
    </div>
    <div id="mlevel9">
      SPWI901 Celestial Host<br />
      SPWI902 Conflagration<br />
      SPWI905 Elysium's Fires<br />
      SPWI909 Power Word: Kill<br />
      SPWI912 Abyssal Fury<br />
      SPWI914 Rune of Torment<br />
    </div>
    <br />
    <br />
    <div id="plevel1">
      SPPR101 Blessing<br />
      SPPR102 Cure Light Wounds<br />
      SPPR103 Curse<br />
      SPPR104 Detect Evil<br />
      SPPR105 Halo of Lesser Revelation<br />
      SPPR106 Protection from Evil<br />
    </div>
    <div id="plevel2">
      SPPR201 Aid<br />
      SPPR202 Cure Moderate Wounds<br />
      SPPR204 Spiritual Hammer<br />
    </div>
    <div id="plevel3">
      SPPR301 Call Lightning<br />
      SPPR302 Flamewalk<br />
      SPPR303 Prayer<br />
      SPPR304 Speak With Dead<br />
      SPPR307 Remove Curse<br />
    </div>
    <div id="plevel4">
      SPPR401 Cure Serious Wounds<br />
    </div>
    <div id="plevel5">
      SPPR501 Cure Critical Wounds<br />
      SPPR502 Raise Dead<br />
    </div>
    <div id="plevel6">
      SPPR601 Heal<br />
    </div>
    <br />
    <br />
    <div id="traps">
      Traps<br />
      SPWI001 Arrow Strike Trap<br />
      SPWI002 Dart Trap<br />
      SPWI003 Spinning Blades Trap<br />
      SPWI004 Avernus Fireball Trap<br />
      SPWI005 Flame Tongue Trap<br />
      SPWI006 Acid Rain Trap<br />
      SPWI007 Unholy Baptism Trap<br />
      SPWI008 Gas Cloud Trap<br />
      SPWI009 Spikes Trap<br />
      SPWI010 Improved Acid Rain Trap<br />
    </div>
    <br />
    <div id="innate">
      Innate<br />
      SPIN101 Litany of Curses<br />
      SPIN102 Skull Mob<br />
      SPIN103 Stories-Bones-Tell
      SPIN104 Sensory Touch<br />
      SPIN106 Fiend Fear Aura<br />
      SPIN107 Kiss (called Kiss of Death in SPELL.IDS)<br />
      SPIN108 Raise Dead<br />
    </div>
    <br />
    <div id="special">
      Special<br />
      SPIN001 Add Litany of Curses<br />
      SPIN002 Removed Litany of Curses<br />
      SPIN003 Add Skull Mob<br />
      SPIN004 Remove Skull Mob<br />
      SPIN005 Add Stories-Bones-Tell<br />
      SPIN006 Remove Stories-Bones-Tell<br />
      SPIN007 Add Sensory Touch<br />
      SPIN008 Remove Sensory Touch<br />
      SPIN009 Add Kiss<br />
      SPIN010 Remove Kiss<br />
      SPIN011 Tint Screen<br />
      SPIN012 Jumble Curse<br />
      SPIN013 Dispel Jumble Curse<br />
      SPIN014 Add Raise Dead<br />
      SPIN015 Remove Raise Dead<br />
    </div>'''

In [344]:
df = pd.DataFrame(re.findall('(?P<SpellCode>\w+\d+)\s*(?P<Spell>.*)', BeautifulSoup(s).text), columns = ['SpellCode', 'Spell'])
df.insert(0, 'Class', df.SpellCode.apply(lambda x: {'SPWI': 'MAGE', 'SPPR':'CLERIC', 'SPIN':'INNATE', 'SPWM': 'WILDMAGE'}.get(re.split('\d+', x)[0])))
                  

In [345]:
df.Spell = df.Spell.apply(lambda x: re.sub('[^0-9a-zA-Z]', '_', x).replace('__', '_').strip('_'))

In [348]:
df.reset_index(drop=True).to_csv(os.path.join(PKG_ROOT, 'resources', 'PSTEE', 'Spell.csv'))

In [296]:
df1 = pd.read_csv(os.path.join(PKG_ROOT, 'resources', 'BGEE', 'HLA.csv'), index_col=0)

In [327]:
pd.concat([df, df1]).drop_duplicates().reset_index().to_csv(os.path.join(PKG_ROOT, 'resources', 'IWD', 'Spell.csv'))

In [326]:
df2 = pd.concat([df, df1])
df2[df2.duplicated()]

,Class,SpellCode,Spell


In [313]:
df[df.duplicated()]

,Class,SpellCode,Spell
375,WILDMAGE,SPWM140,Drain_Item_Charges


In [336]:
df = pd.read_csv(os.path.join(PKG_ROOT, 'resources', 'BGEE', 'Spell.csv'), index_col=0)

In [338]:
df.drop('index',axis=1).reset_index(drop=True).to_csv(os.path.join(PKG_ROOT, 'resources', 'BGEE', 'Spell.csv'))

In [349]:
PKG_ROOT

'/Users/CIDER/工作/Python程序/游戏/博德之门/New'